# regional_fuel_market, zone_to_regional_fuel_market and fuel_supply_curves

In [1]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

import pandas as pd
import numpy as np

###
# module issue
import os
import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
###
from pathlib import Path
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)

cwd = Path.cwd()

settings_path = (cwd / "settings_TD.yml")
settings = load_settings(settings_path)

In [2]:
# read in csv outputs from PG
#existing_gen = pd.read_csv('PG_output_csv/existing_gen_WECC.csv', index_col=0)
#new_gen = pd.read_csv('PG_output_csv/new_gen_WECC.csv', index_col=0)
#existing_variability = pd.read_csv('PG_output_csv/existing_variability_WECC.csv', index_col=0)
#potential_build_yr = pd.read_csv('PG_output_csv/gc_units_model_WECC.csv', index_col=0)
# all_gen = pd.read_csv('PG_output_csv/all_gen_WECC.csv', index_col=0)
# fuels = pd.read_csv('PG_output_csv/fuels_WECC.csv', index_col=0)
fuel_prices = pd.read_csv('PG_output_csv/fuel_prices_WECC.csv', index_col=0)
#load_curves = pd.read_csv('PG_output_csv/load_curves_WECC.csv', index_col=0)

In [3]:
aeo_fuel_region_map = settings.get('aeo_fuel_region_map')

# regional_fuel_market and zone_to_regional_fuel_market

In [4]:
def fuel_market_tables(fuel_prices, aeo_fuel_region_map, scenario):
    '''
    Create regional_fuel_markets and zone_to_regional_fuel_market
    SWITCH does not seem to like this overlapping with fuel_cost. So all of this might be incorrect.
    '''
    
    # create initial regional fuel market.  Format: region - fuel
    reg_fuel_mar_1 = fuel_prices.copy()
    reg_fuel_mar_1 = reg_fuel_mar_1.loc[reg_fuel_mar_1['scenario']==scenario] # use reference for now
    reg_fuel_mar_1 = reg_fuel_mar_1.drop(['year','price','full_fuel_name','scenario'], axis=1)
    reg_fuel_mar_1 = reg_fuel_mar_1.rename(columns={'region':'regional_fuel_market'})
    reg_fuel_mar_1 = reg_fuel_mar_1[['regional_fuel_market', 'fuel']]
    
    fuel_markets = reg_fuel_mar_1['regional_fuel_market'].unique()
    
    # region to fuel
    group = reg_fuel_mar_1.groupby('regional_fuel_market')
    fuel_market_dict = {}
    for region in fuel_markets:
        df = group.get_group(region)
        fuel = df['fuel'].to_list()
        fuel = list(set(fuel))
        fuel_market_dict[region] = fuel
    
    # create zone_regional_fuel_market
    data = list()
    for region in aeo_fuel_region_map.keys():
        for i in range(len(aeo_fuel_region_map[region])):
            ipm = aeo_fuel_region_map[region][i]
            for fuel in fuel_market_dict[region]:
                data.append([ipm, ipm +'-'+ fuel])

    zone_regional_fm = pd.DataFrame(data, columns=['load_zone', 'regional_fuel_market'])
    
    # use that to finish regional_fuel_markets
    regional_fuel_markets = zone_regional_fm.copy()
    regional_fuel_markets['fuel_list'] = regional_fuel_markets['regional_fuel_market'].str.split('-')
    regional_fuel_markets['fuel'] = regional_fuel_markets['fuel_list'].apply(lambda x: x[-1])
    regional_fuel_markets = regional_fuel_markets[['regional_fuel_market','fuel']]
    
    
    return regional_fuel_markets, zone_regional_fm

In [5]:
rfm, zrfm = fuel_market_tables(fuel_prices, aeo_fuel_region_map, scenario='reference')

In [6]:
rfm # there can't be overlap with fuel_cost. So this table isn't right

,regional_fuel_market,fuel
0,WECC_AZ-coal,coal
1,WECC_AZ-uranium,uranium
2,WECC_AZ-naturalgas,naturalgas
3,WECC_AZ-distillate,distillate
4,WECC_NM-coal,coal
...,...,...
267,NENGREST-distillate,distillate
268,NENG_CT-coal,coal
269,NENG_CT-uranium,uranium
270,NENG_CT-naturalgas,naturalgas


In [7]:
zrfm # there can't be overlap with fuel_cost. So this table isn't right

,load_zone,regional_fuel_market
0,WECC_AZ,WECC_AZ-coal
1,WECC_AZ,WECC_AZ-uranium
2,WECC_AZ,WECC_AZ-naturalgas
3,WECC_AZ,WECC_AZ-distillate
4,WECC_NM,WECC_NM-coal
...,...,...
267,NENGREST,NENGREST-distillate
268,NENG_CT,NENG_CT-coal
269,NENG_CT,NENG_CT-uranium
270,NENG_CT,NENG_CT-naturalgas


# Running inputs in SWITCH
SWITCH does not seem to like the two fuel market tables above to overlap with fuel_cost. REAM has separate fuels in regional_fuel_markets and fuel_cost (bio_solid and bio_gas) The two dummy tables below makes the error go away but that probably isn't correct

fuels in fuel_cost + fuels in regional_fuel_markets = fuels in the fuels table

SWITCH also doesn't like not having a fuel_supply_curve table.  Creating a dummy table here based on region_fuel_market

In [8]:
regional_fuel_markets = pd.DataFrame({'regional_fuel_market':'loadzone-Fuel','fuel':'Fuel'}, index=[0])
regional_fuel_markets

,regional_fuel_market,fuel
0,loadzone-Fuel,Fuel


In [9]:
zone_regional_fm = pd.DataFrame({'load_zone':'loadzone','fuel':'loadzone-Fuel'}, index=[0])
zone_regional_fm

,load_zone,fuel
0,loadzone,loadzone-Fuel


# fuel supply curves

In [10]:
# creating dummy values based on one load zone in REAM's input file
# regional_fuel_market should align with the regional_fuel_market table
fuel_supply_curves20 = pd.DataFrame({'period':[2020,2020,2020,2020,2020,2020], 'tier':[1,2,3,4,5,6],
                                     'unit_cost':[1.9, 4.0, 487.5, 563.7, 637.8, 816.7], 
                                     'max_avail_at_cost':[651929, 3845638,3871799,3882177,3889953,3920836]})
fuel_supply_curves20.insert(0,'regional_fuel_market','loadzone-Fuel')
fuel_supply_curves30 = fuel_supply_curves20.copy()
fuel_supply_curves30['period'] = 2030
fuel_supply_curves40 = fuel_supply_curves20.copy()
fuel_supply_curves40['period'] = 2040
fuel_supply_curves50 = fuel_supply_curves20.copy()
fuel_supply_curves50['period'] = 2050
fuel_supply_curves = pd.concat([fuel_supply_curves20, fuel_supply_curves30, fuel_supply_curves40,fuel_supply_curves50])
fuel_supply_curves

,regional_fuel_market,period,tier,unit_cost,max_avail_at_cost
0,loadzone-Fuel,2020,1,1.9,651929
1,loadzone-Fuel,2020,2,4.0,3845638
2,loadzone-Fuel,2020,3,487.5,3871799
3,loadzone-Fuel,2020,4,563.7,3882177
4,loadzone-Fuel,2020,5,637.8,3889953
5,loadzone-Fuel,2020,6,816.7,3920836
0,loadzone-Fuel,2030,1,1.9,651929
1,loadzone-Fuel,2030,2,4.0,3845638
2,loadzone-Fuel,2030,3,487.5,3871799
3,loadzone-Fuel,2030,4,563.7,3882177


In [11]:
regional_fuel_markets.to_csv(r'SWITCH_Inputs\regional_fuel_markets.csv', index = False)
zone_regional_fm.to_csv(r'SWITCH_Inputs\zone_to_regional_fuel_market.csv', index = False)
fuel_supply_curves.to_csv(r'SWITCH_Inputs\fuel_supply_curves.csv', index = False)